In [1]:
!pip install -q -U trl pyarrow==14.0.1 requests==2.31.0 transformers accelerate peft datasets bitsandbytes einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━

In [2]:
from datasets import load_dataset

dataset_name = 'NeelkanthKhithani/ethical-scenarios-llama-3'
dataset = load_dataset(dataset_name, split="train")

Generating train split:   0%|          | 0/500 [00:00<?, ? examples/s]

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from google.colab import userdata

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    token=userdata.get('HF_TOKEN')
)
model.config.use_cache = False

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, token=userdata.get('HF_TOKEN'))
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
torch.cuda.empty_cache()

In [6]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.01
lora_r = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [12]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [13]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [14]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [15]:
torch.cuda.empty_cache()

In [17]:
trainer.train()

Step,Training Loss
10,2.261200
20,1.531400
30,1.095200
40,1.069600
50,1.063700
60,0.989000
70,1.023200
80,0.974900
90,0.917100
100,0.973300


TrainOutput(global_step=100, training_loss=1.189862241744995, metrics={'train_runtime': 527.3038, 'train_samples_per_second': 0.759, 'train_steps_per_second': 0.19, 'total_flos': 5875480971018240.0, 'train_loss': 1.189862241744995, 'epoch': 0.8})

In [20]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
model_to_save.save_pretrained("outputs")
tokenizer.save_pretrained("outputs")

('outputs/tokenizer_config.json',
 'outputs/special_tokens_map.json',
 'outputs/tokenizer.json')

In [21]:
config = model_to_save.config
config.save_pretrained("outputs")

In [22]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [33]:
prompt = '''
Generate a detailed scenario in hindi focusing on the theme of "Ethical and Moral Education".
The scenario should present a situation where Ethical and Moral Education is significantly tested or clearly demonstrated.
Ensure that the context is relatable and the characters' emotions and motivations are well-explored.

Following the scenario, generate exactly two decision questions in hindi that prompt the students to consider both a positive and a negative response to the situation.

*Important Note: Provide only one hindi scenario and two hindi decision questions. Do not generate multiple scenarios.
**Please Ensure the entire response is within 250 tokens.*

*Give the output in this specific way everytime*:

Scenario: ...
Decision 1: ...?
Decision 2: ...?
End
'''
device = "cuda:0"

inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=400)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Generate a detailed scenario in hindi focusing on the theme of "Ethical and Moral Education".
The scenario should present a situation where Ethical and Moral Education is significantly tested or clearly demonstrated.
Ensure that the context is relatable and the characters' emotions and motivations are well-explored.

Following the scenario, generate exactly two decision questions in hindi that prompt the students to consider both a positive and a negative response to the situation.

*Important Note: Provide only one hindi scenario and two hindi decision questions. Do not generate multiple scenarios. 
**Please Ensure the entire response is within 250 tokens.*

*Give the output in this specific way everytime*:

Scenario:...
Decision 1:...?
Decision 2:...?
End
**End of Response**

Scenario:
एक छोटे से शहर में एक स्कूल है, जहाँ एक छात्र, राहुल, है, जो अपने स्कूल के मुख्यालय में कार्य करता है। एक दिन, राहुल के स्कूल के मुख्यालय में एक प्राचल की मृत्यु हो जाती है। प्राचल की मृत्यु के बाद, स

In [27]:
from huggingface_hub import login
login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [28]:
model.push_to_hub("moral-mentor-llama-3")
tokenizer.push_to_hub("moral-mentor-llama-3")

adapter_model.safetensors:   0%|          | 0.00/54.5M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/NeelkanthKhithani/moral-mentor-llama-3/commit/7b81144caac18e957fbb5000dce51b67da8eab75', commit_message='Upload tokenizer', commit_description='', oid='7b81144caac18e957fbb5000dce51b67da8eab75', pr_url=None, pr_revision=None, pr_num=None)